In [1]:
import random
import os
import pandas as pd
import numpy as np
import copy
from IPython.core.debugger import set_trace
import pixiedust

Pixiedust database opened successfully


Table service_connections created successfully


In [2]:
with open('full_dict_relations.txt', 'r') as f:
    full_dict_relations = eval(f.read())

In [4]:
def best_person(group, last_people, remove = True):
    mid_dict = {}
    for i in last_people:
        sum_1_to_others = 0
        for j in list_groups[group - 1]:
            try:
                sum_1_to_others += full_dict_relations[i][j]
            except:
                pass
            try:
                sum_1_to_others += full_dict_relations[j][i]
            except:
                pass
        mid_dict[i] = sum_1_to_others
    sorted_dict = {k: v for k, v in sorted(mid_dict.items(), key=lambda item: item[1], reverse = True)}
    a = list(sorted_dict.keys())[0]
    if remove:
        last_people.remove(a)
#         print(a)
    return a

In [5]:
# получает номер группы, возвращает сумму
def sum_relation(group, list_groups):
    all_sum = 0
    for i in list_groups[group - 1]:
        for j in list_groups[group - 1]:
            try:
                all_sum += full_dict_relations[i][j]
            except:
                pass
            try:
                all_sum += full_dict_relations[j][i]
            except:
                pass
    return all_sum

In [6]:
# получает список групп, возвращает общую сумму и численность всех групп в сумме.
def sum_all_groups(list_groups):
    all_sum = 0
    population = 0
    for i in range(1, 7):
        population += len(list_groups[i - 1])
        all_sum += sum_relation(i, list_groups)
    return all_sum, population

In [7]:
# получает ключ человека в словаре отношений и номер группы.
# возвращает человека и его вклад в цифрах.
def contribution_one_person(person, num_group):
    sum_contr = 0
    for i in list_groups[num_group - 1]:
        try:
            sum_contr += full_dict_relations[person][i]
        except:
            pass
        try:
            sum_contr += full_dict_relations[i][person]
        except:
            pass
    return (person, sum_contr)

In [3]:
# reset
list_groups = [[],[],[],[],[],[]]
last_people_base = list(full_dict_relations.keys())
last_people = list(full_dict_relations.keys())

In [646]:
# найти шесть лучших пятнадцаток.

In [11]:
for i in range(6):
    dict_10_groups = {}
    for k in last_people_base[:]:     
        list_groups[i] = []
        last_people = last_people_base[:]
        list_groups[i].append(k)
        last_people.remove(k)
        for j in range(14):
            list_groups[i].append(best_person(i + 1, last_people, remove = True))
        a = sum_relation(i + 1, list_groups = list_groups)
        dict_10_groups[a] = list_groups[i] 
    list_groups[i] = [] #зачем?
    list_groups[i] = dict_10_groups[max(dict_10_groups)]

    for people in list_groups[i]:
        last_people_base.remove(people)

In [12]:
sum_all_groups(list_groups = list_groups)

(2648.0, 90)

In [5]:
# их заполнить с перетасовкой

In [8]:
# дать группу человека, его имя и коэф для тюна. Возвращает список рациональных переходов для этого человека.
def allowed_groups(group, name, coeff_min):
    _, current_contr = contribution_one_person(name, group)
    minim_contrib = current_contr / coeff_min
    list_allowed_groups = []
    for i in range(6):
        _, contr = contribution_one_person(name, i + 1)
        if contr > minim_contrib or current_contr == 0:
            list_allowed_groups.append(i + 1)
    if group not in list_allowed_groups:
        list_allowed_groups.append(group)
    return list_allowed_groups

In [9]:
# дать словарь возможных комбинация. Возвращает количество комбинаций.
def num_comb(dictionary):
    list_comb = []
    n_comb = 1
    for key, value in dictionary.items():
        n_comb = n_comb * len(value)
    return n_comb

In [54]:
# работает неплохо. Но группы могут быть произвольной длины. И кодом пытался это предотвратить, выдавал ошибку.
# дать словарь с возможными комбинациями и список групп. Он выдаст новую комбинацию.
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    print(n_comb)
    dict_comb = {}
    ground_list_groups = copy.deepcopy(groups)
    for i in range(n_comb):
        ground_list_groups = copy.deepcopy(groups)
        k = 0
        for key, value in dictionary.items():
            if len(value) > 1 and k == 0:
                ground_list_groups[value.pop() - 1].append(key)
                k += 1
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(ground_list_groups)
        print(sum_gr)
        dict_comb[sum_gr] = copy.deepcopy(ground_list_groups)
        
    print(len(dict_comb))  
#     dict1 = copy.deepcopy(dict_comb)
#     for key, values in dict_comb.items():
#         for i in values:
#             if len(i) > 30:
#                 del dict1[key]
    return dict_comb[max(dict_comb)]   

In [56]:
# пытался поменять ключ со значением. Плохо получилось.
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    print(n_comb)
    dict_comb = {}
    ground_list_groups = copy.deepcopy(groups)
    for i in range(n_comb):
        k = 0
        ground_list_groups = copy.deepcopy(groups)
        for key, value in dictionary.items():
            if len(value) > 1 and k == 0:
                ground_list_groups[value.pop() - 1].append(key)
                k += 1
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(list_groups = ground_list_groups)
        dict_comb[str(ground_list_groups)] = sum_gr
#         set_trace()
    print(len(dict_comb))   
#     dict1 = copy.deepcopy(dict_comb)
#     for key, values in dict_comb.items():
#         for i in eval(key):
#             if len(i) > 30:
#                 del dict1[key]
    return eval(max(dict_comb, key=dict_comb.get))

In [437]:
# dict_change_groups

In [15]:
depth = 3
coeff_min = 2
# n_group_for_free = 3

In [27]:
len(last_people)

84

In [ ]:
# делаю лишние копии, потому что у меня уже сдвиг на связанных списках.

In [28]:
while len(last_people) > 0:
#     pull_students = []
    dict_min = {}
    for group in range(6):
        dict_rel = {}
        for i in list_groups[group]:
            name, contr = contribution_one_person(i, group + 1)
            dict_rel[name] = contr
        list_min = []
        for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
            list_min.append(i[0])
        dict_min[group + 1] = list_min
        for i in list_min:
            list_groups[group].remove(i)
#         pull_students.append(last_min)
    dict_change_groups = {}
    for key, value in dict_min.items():
        for i in value:
            dict_change_groups[i] = allowed_groups(key, i, coeff_min)
    free = last_people.pop()
#     pull_students.append(free)
    dict_change_groups[free] = list(range(1, 7))
#     for i in pull_students:
#     n_comb = num_comb(dict_change_groups)
    new_list_groups = copy.deepcopy(best_combination(dict_change_groups, copy.deepcopy(list_groups)))
    all_sum, pop = sum_all_groups(list_groups = new_list_groups)
    print(all_sum, pop, all_sum / pop)
    list_groups = copy.deepcopy(new_list_groups)
    

2650.0 91 29.12087912087912
2654.0 92 28.847826086956523
2679.0 93 28.806451612903224
2687.0 94 28.585106382978722
2691.0 95 28.326315789473686
2706.0 96 28.1875
2718.0 97 28.02061855670103
2737.0 98 27.928571428571427
2754.0 99 27.818181818181817
2786.0 100 27.86
2818.0 101 27.900990099009903
2871.0 102 28.147058823529413
2906.0 103 28.21359223300971
2938.0 104 28.25
2972.0 105 28.304761904761904
3033.0 106 28.61320754716981
3072.0 107 28.710280373831775
3100.0 108 28.703703703703702
3161.0 109 29.0
3188.0 110 28.98181818181818
3188.0 110 28.98181818181818
3272.0 111 29.47747747747748
3353.0 112 29.9375
3422.0 113 30.283185840707965
3466.0 114 30.403508771929825
3538.0 115 30.765217391304347
3591.0 116 30.95689655172414
3641.0 117 31.11965811965812
3715.0 118 31.483050847457626
3769.0 119 31.672268907563026
3833.0 120 31.941666666666666
3917.0 121 32.37190082644628
3987.0 122 32.68032786885246
4049.0 123 32.91869918699187
4139.0 124 33.37903225806452
4254.0 125 34.032
4320.0 126 34.28

In [57]:
# %%pixie_debugger
dict_min = {}
for group in range(6):
    dict_rel = {}
    for i in list_groups[group]:
        name, contr = contribution_one_person(i, group + 1)
        dict_rel[name] = contr
    list_min = []
    for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
        list_min.append(i[0])
    dict_min[group + 1] = list_min
    for i in list_min:
        list_groups[group].remove(i)

dict_change_groups = {}
for key, value in dict_min.items():
    for i in value:
        dict_change_groups[i] = allowed_groups(key, i, coeff_min)

free = last_people.pop()
dict_change_groups[free] = list(range(1, 7))
# print(dict_change_groups)
new_list_groups = copy.deepcopy(best_combination(dict_change_groups, copy.deepcopy(list_groups)))
all_sum, pop = sum_all_groups(list_groups = new_list_groups)
print(all_sum, pop, all_sum / pop)
list_groups = copy.deepcopy(new_list_groups)

72
[['Amos Finley', 'Hermelinda Kamer', 'Aleisha Cherry', 'Neal Rustin', 'German Shelton', 'Sina Maggio', 'Armand Valencia', 'Michale Anthony', 'Peggy Menter', 'Mack Geyer', 'Kathie Moses', 'Jay Luper', 'Thomas Munch', 'Barrie Youmans', 'Leonor Simons', 'Ruby Lester'], ['Houston Leblanc', 'Belia Avey', 'Stacy Wilkinson', 'Nanette Shepherd', 'Joann Huynh', 'Dewayne Kaiser', 'Keenan Stewart', 'Klara Pagani', 'Cris Brundidge', 'Mandi Sponsler', 'Tracey Duarte', 'Marinda Mccotter', 'Genevive Calle', 'Hosea Kelley', 'Gaston Fitts'], ['Aaron Sharp', 'Felix Laursen', 'Franklyn Palmer', 'Marlin Burns', 'Shanice Kriss', 'Jeannine Valone', 'Edison Nelson', 'Marcus Quinley', 'Miguel Andres', 'Effie Mercado', 'Alan Franklin', 'Aileen Kramer', 'Mathew Livingston', 'Eryn Knick', 'Landon Mason', 'Trudy Barber', 'Krystal Martin', 'Alta Schroeder', 'Avery Petrus', 'Marth Valcourt', 'Freddy Gosier', 'Florencio Pittman', 'Pok Elsasser', 'Ludie Gee', 'Rosalinda Boren', 'Corinne Gebo', 'Lincoln Good', 'Ely

[['Amos Finley', 'Hermelinda Kamer', 'Aleisha Cherry', 'Neal Rustin', 'German Shelton', 'Sina Maggio', 'Armand Valencia', 'Michale Anthony', 'Peggy Menter', 'Mack Geyer', 'Kathie Moses', 'Jay Luper', 'Thomas Munch', 'Leland Harris', 'Barrie Youmans', 'Leonor Simons', 'Ruby Lester'], ['Houston Leblanc', 'Belia Avey', 'Stacy Wilkinson', 'Nanette Shepherd', 'Joann Huynh', 'Dewayne Kaiser', 'Keenan Stewart', 'Klara Pagani', 'Cris Brundidge', 'Mandi Sponsler', 'Tracey Duarte', 'Marinda Mccotter', 'Genevive Calle', 'Hosea Kelley', 'Gaston Fitts'], ['Aaron Sharp', 'Felix Laursen', 'Franklyn Palmer', 'Marlin Burns', 'Shanice Kriss', 'Jeannine Valone', 'Edison Nelson', 'Marcus Quinley', 'Miguel Andres', 'Effie Mercado', 'Alan Franklin', 'Aileen Kramer', 'Mathew Livingston', 'Eryn Knick', 'Landon Mason', 'Trudy Barber', 'Krystal Martin', 'Alta Schroeder', 'Avery Petrus', 'Marth Valcourt', 'Freddy Gosier', 'Florencio Pittman', 'Pok Elsasser', 'Ludie Gee', 'Rosalinda Boren', 'Corinne Gebo', 'Linco

In [90]:
free

'Herschel Neifert'

In [81]:
dict_change_groups['huina'] = list(range(1, 7))

In [92]:
dict_change_groups

{'Stacie Gillespie': [1],
 'Barrie Youmans': [1],
 'Leonor Simons': [1],
 'Genevive Calle': [2],
 'Hosea Kelley': [2],
 'Gaston Fitts': [2],
 'Sherly Candela': [2],
 'Herschel Neifert': [3],
 'Trudy Barber': [3],
 'Deeanna Wiers': [4],
 'Alton Edelson': [4],
 'Dario Grimes': [4],
 'Abraham Callan': [5],
 'Errol Fox': [5],
 'Cora Gautier': [5],
 'Bao Wilde': [6],
 'Kristi Meunier': [6],
 'Trena Sebree': [1],
 'Ludie Gee': [1]}

In [85]:
len(last_people)

80

In [76]:
dict_min

{1: ['Jay Luper', 'Barrie Youmans', 'Sina Maggio'],
 2: ['Thomas Munch', 'Genevive Calle', 'Hosea Kelley'],
 3: ['Aileen Kramer', 'Mathew Livingston', 'Eryn Knick'],
 4: ['Deeanna Wiers', 'Alton Edelson', 'Stacie Gillespie'],
 5: ['Abraham Callan', 'Errol Fox', 'Jovita Quattlebaum'],
 6: ['Trudy Barber', 'Cora Gautier', 'Bao Wilde']}

In [64]:
dict_change_groups

{'Barrie Youmans': [1],
 'Leonor Simons': [1],
 'Thomas Munch': [1],
 'Genevive Calle': [2],
 'Hosea Kelley': [2],
 'Gaston Fitts': [2],
 'Aileen Kramer': [3],
 'Eryn Knick': [3],
 'Landon Mason': [3],
 'Deeanna Wiers': [4],
 'Alton Edelson': [4],
 'Stacie Gillespie': [1],
 'Abraham Callan': [5],
 'Errol Fox': [5],
 'Cora Gautier': [5],
 'Bao Wilde': [6],
 'Kristi Meunier': [6],
 'Trena Sebree': [1],
 'Donnell Barnett': [1]}

In [48]:
list_groups[5]

['Ruby Lester',
 'Clay Powell',
 'Rory Holmes',
 'Julissa Riggins',
 'William Blind',
 'Manuela George',
 'Earline Mcpherson',
 'Erna Krueger',
 'Bobby Bowers',
 'Shanae Stoughton',
 'Chas French',
 'Rosie Watkins',
 'Bao Wilde',
 'Kristi Meunier',
 'Trena Sebree']

In [51]:
for i in list_groups:
    print(len(i))

16
15
35
15
15
28


In [332]:
contribution_one_person('Aileen Kramer', 2)

('Aileen Kramer', 0)

In [140]:
dict_min = {}
for group in range(6):
    dict_rel = {}
    for i in list_groups[group]:
        name, contr = contribution_one_person(i, 1)
        dict_rel[name] = contr
    list_min = []
    for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
        list_min.append(i[0])
    dict_min[group + 1] = list_min
dict_change_groups = {}
for key, value in dict_min.items():
    for i in value:
        dict_change_groups[i] = allowed_groups(key, i, coeff_min)
free = last_people[0]
dict_change_groups[free] = list(range(1, 7))

In [524]:
dict_change_groups

{'Mary Clements': [1],
 'Reynaldo Bruce': [1],
 'Laurence Gapinski': [1],
 'Lincoln Good': [2],
 'Jeannine Valone': [2],
 'Israel Arreguin': [2],
 'Amos Finley': [3],
 'Sina Maggio': [3],
 'Emile Ferguson': [3],
 'Pearl Lloyd': [4],
 'Deshawn Tower': [4],
 'Sandra Spradling': [4],
 'Peggie Brent': [5],
 'Katrina Tocco': [5],
 'Jamie Mayer': [5],
 'Clay Powell': [6],
 'Rosie Watkins': [6],
 'Darell Mcglown': [6],
 'Thomas Munch': [1]}

In [210]:
dict1 = {}
for i in list_groups[0]:
    name, contr = contribution_one_person(i, 1)
    dict1[name] = contr

In [258]:
allowed_groups(1, 'Eryn Knick', coeff_min = 2)

[3, 5]

In [226]:
for i in sorted(dict1.items(), key=lambda item: item[1])[:4]:
    print(i[0])

Tracey Duarte
Cris Brundidge
Klara Pagani
Keenan Stewart


In [218]:
sorted(dict1.items(), key=lambda item: item[1])[:5]

[('Tracey Duarte', 23.5),
 ('Cris Brundidge', 23.5),
 ('Klara Pagani', 23.5),
 ('Keenan Stewart', 28.0),
 ('Dewayne Kaiser', 28.0),
 ('Stacy Wilkinson', 28.0),
 ('Joann Huynh', 30.5),
 ('Belia Avey', 30.5),
 ('Mandi Sponsler', 33.0),
 ('Nanette Shepherd', 37.5)]

In [213]:
dict1

{'Mandi Sponsler': 33.0,
 'Tracey Duarte': 23.5,
 'Cris Brundidge': 23.5,
 'Klara Pagani': 23.5,
 'Keenan Stewart': 28.0,
 'Dewayne Kaiser': 28.0,
 'Joann Huynh': 30.5,
 'Nanette Shepherd': 37.5,
 'Belia Avey': 30.5,
 'Stacy Wilkinson': 28.0}

In [ ]:
# посчитать общую сумму.